In this lecture we're going to talk about querying DataFrames. The first step in the process is to understand 
Boolean masking. Boolean masking is the heart of fast and efficient querying in numpy and pandas, and its 
analogous to bit masking used in other areas of computational science. By the end of this lecture you'll 
understand how Boolean masking works, and how to apply this to a DataFrame to get out data you're interested in.

A Boolean mask is an array which can be of one dimension like a series, or two dimensions like a data frame, 
where each of the values in the array are either true or false. This array is essentially overlaid on top 
of the data structure that we're querying. And any cell aligned with the true value will be admitted into 
our final result, and any cell aligned with a false value will not.

In [2]:
# Let's start with an example and import our graduate admission dataset. 
# First we'll bring in pandas
import pandas as pd
# Then we'll load in our CSV file
df = pd.read_csv('datasets/Admission_Predict.csv', index_col=0)
# And we'll clean up a couple of poorly named 
# columns like we did in a previous lecture
df.columns = [x.lower().strip() for x in df.columns]
# And we'll take a look at the results
df.head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
# Boolean masks are created by applying operators directly 
# to the pandas Series or DataFrame objects. 
# For instance, in our graduate admission dataset,
# we might be interested in seeing only those students 
# that have a chance higher than 0.7

# To build a Boolean mask for this query,
# we want to project the chance of admit column using the 
# indexing operator and apply the greater than operator with a 
# comparison value of 0.7. This is 
# essentially broadcasting a comparison operator, greater than, 
# with the results being returned as 
# a Boolean Series. 
# The resultant Series is indexed where the value of 
# each cell is either True or False 
# depending on whether a student has a chance of admit higher than 0.7
admit_mask=df['chance of admit'] > 0.7
admit_mask

Serial No.
1       True
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398     True
399    False
400     True
Name: chance of admit, Length: 400, dtype: bool

In [3]:
# This is pretty fundamental, so take a moment to look at this. 
# The result of broadcasting a comparison operator is a Boolean mask
# - true or false values depending upon the results of the comparison. 
#Underneath, pandas is applying the comparison operator you specified 
# through vectorization (so efficiently and in
# parallel) to all of the values in the array you specified which,
#in this case, is the chance of admit column of the dataframe.
# The result is a series, since only one column is being operator on, 
# filled with either True or False values,
#which is what the comparison operator returns.

In [4]:
# So, what do you do with the boolean mask once you have formed it?
# Well, you can just lay it on top of the
# data to "hide" the data you don't want, 
# which is represented by all of the False values. We do this by using
# the .where() function on the original DataFrame.
df.where(admit_mask).head(10)

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,330.0,115.0,5.0,4.5,3.0,9.34,1.0,0.90
7,321.0,109.0,3.0,3.0,4.0,8.20,1.0,0.75
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# We see that the resulting data frame keeps the original indexed values, 
# and only data which met the condition was retained. 
# All of the rows which did not meet the condition have NaN data instead,
# but these rows were not dropped from our dataset. 
#
# The next step is, if we don't want the NaN data,
# we use the dropna() function
df.where(admit_mask).dropna().head(10)

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
6,330.0,115.0,5.0,4.5,3.0,9.34,1.0,0.90
7,321.0,109.0,3.0,3.0,4.0,8.20,1.0,0.75
12,327.0,111.0,4.0,4.0,4.5,9.00,1.0,0.84
13,328.0,112.0,4.0,4.0,4.5,9.10,1.0,0.78
23,328.0,116.0,5.0,5.0,5.0,9.50,1.0,0.94


In [11]:
# The returned DataFrame now has all of the NaN rows dropped. 
# Notice the index now includes one through four and six, but not five.
#
# Despite being really handy, where() isn't actually used that often. 
# Instead, the pandas devs created a shorthand syntax which combines where() 
# and dropna(), doing both at once. And, in
# typical fashion, the just overloaded the indexing operator to do this!

df[df['chance of admit'] > 0.7].head(10)

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
6,330,115,5,4.5,3.0,9.34,1,0.90
7,321,109,3,3.0,4.0,8.20,1,0.75
12,327,111,4,4.0,4.5,9.00,1,0.84
13,328,112,4,4.0,4.5,9.10,1,0.78
23,328,116,5,5.0,5.0,9.50,1,0.94


In [12]:
# I personally find this much harder to read, 
# but it's also very more common when you're reading other
# people's code, so it's important to be able to understand it. 
# Just reviewing this indexing operator on DataFrame, it now does two things:

# It can be called with a string parameter to project a single column
df["gre score"].head(10)

Serial No.
1     337
2     324
3     316
4     322
6     330
7     321
12    327
13    328
23    328
24    334
Name: gre score, dtype: int64

In [13]:
# Or you can send it a list of columns as strings
df[["gre score","toefl score"]].head()

,gre score,toefl score
Serial No.,,
1,337,118
2,324,107
3,316,104
4,322,110
6,330,115


In [18]:
# Or you can send it a boolean mask
df[df["gre score"]>335].head(100)

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
25,336,119,5,4.0,3.5,9.80,1,0.97
26,340,120,5,4.5,4.5,9.60,1,0.94
33,338,118,4,3.0,4.5,9.40,1,0.91
34,340,114,5,4.0,4.0,9.60,1,0.90
48,339,119,5,4.5,4.0,9.70,0,0.89
72,336,112,5,5.0,5.0,9.76,1,0.96
82,340,120,4,5.0,5.0,9.50,1,0.96
85,340,115,5,4.5,4.5,9.45,1,0.94


In [13]:
# And each of these is mimicing functionality from either 
# .loc() or .where().dropna().

In [19]:
# Before we leave this, lets talk about combining multiple boolean masks,
# such as multiple criteria for including.
# In bitmasking in other places in computer science this is done with "and",
# if both masks must be
# True for a True value to be in the final mask),
#or "or" if only one needs to be True.

# Unfortunatly, it doesn't feel quite as natural in pandas.
# For instance, if you want to take two boolean
# series and and them together
(df['chance of admit'] > 0.7) and (df['chance of admit'] < 0.9)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [22]:
# This doesn't work. And despite using pandas for awhile, 
# I still find I regularly try and do this.
#The problem is that you have series objects,
# and python underneath doesn't know how to compare two series using
# and or or. 
#Instead, the pandas authors have overwritten the pipe | and ampersand &
#operators to handle this for us
(df['chance of admit'] > 0.7) & (df['chance of admit'] < 0.9)

Serial No.
1      False
2       True
3       True
4       True
6      False
       ...  
395     True
396     True
397     True
398    False
400    False
Name: chance of admit, Length: 235, dtype: bool

In [23]:
# One thing to watch out for is order of operations!
# A common error for new pandas users is
# to try and do boolean comparisons using the & operator 
#but not putting parentheses around
# the individual terms you are interested in
df['chance of admit'] > 0.7 & df['chance of admit'] < 0.9

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

In [17]:

#The problem is that Python is trying to bitwise and a 0.7 
# and a pandas dataframe, when you really want
# to bitwise and the broadcasted dataframes together

SyntaxError: invalid syntax (<ipython-input-17-179fe609bdb2>, line 1)

In [18]:
# Another way to do this is to just get rid of the comparison 
# operator completely, and instead
# use the built in functions which mimic this approach
df['chance of admit'].gt(0.7) & df['chance of admit'].lt(0.9)

Serial No.
1      False
2       True
3       True
4       True
5      False
6      False
7       True
8      False
9      False
10     False
11     False
12      True
13      True
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27      True
28     False
29     False
30     False
       ...  
371     True
372     True
373    False
374     True
375    False
376    False
377    False
378    False
379    False
380     True
381     True
382     True
383     True
384    False
385    False
386    False
387    False
388    False
389    False
390     True
391    False
392     True
393     True
394     True
395     True
396     True
397     True
398    False
399    False
400    False
Name: chance of admit, Length: 400, dtype: bool

In [24]:
# These functions are build right into the Series and DataFrame objects, 
# so you can chain them
# too, which results in the same answer and the use of no visual operators.
#You can decide what
# looks best for you
df['chance of admit'].gt(0.7).lt(0.9)

Serial No.
1      False
2      False
3      False
4      False
6      False
       ...  
395    False
396    False
397    False
398    False
400    False
Name: chance of admit, Length: 235, dtype: bool

In [20]:
# This only works if your operator, such as less than or greater than,
# is built into the DataFrame, but I
# certainly find that last code example much more readable
# than one with ampersands and parenthesis.

In [25]:
# You need to be able to read and write all of these, 
# and understand the implications of the route you are
 # choosing. It's worth really going back and rewatching 
    #this lecture to make sure you have it. I would say
 # 50% or more of the work you'll be doing 
# in data cleaning involves querying DataFrames.